In [14]:
# !pip install -U datasets

In [8]:
# !sudo -H pip install -q transformers --upgrade

In [15]:
# !pip uninstall autoawq -y

In [16]:
# !pip install autoawq

In [17]:
# !sudo -H pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp38-cp38-linux_x86_64.whl

In [18]:
# !pip install transformers==4.35.2

In [1]:
import pandas
import numpy
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer,AutoModelForCausalLM

In [3]:
import torch
torch.__version__

'2.0.1+cu117'

In [8]:
#model_path = '/data/mistral/query-to-mql/exp-10/nov-20/merged-model'

In [14]:
quant_path = '/llm/quantized_model/awq'

In [10]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [4]:
model_path='lmsys/vicuna-7b-v1.5'

In [5]:
quant_config = {"zero_point":True,
               "q_group_size": 128,
               "w_bit": 4,
               "version": "GEMM"
               }

In [6]:
#load model
# model = AutoAWQForCausalLM.from_pretrained(model_path, **{"low_cpu_mem_usage":True})
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map='auto')

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/tmp/pip_packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/tmp/pip_packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [8]:
import torch
torch.cuda.empty_cache()

In [9]:
import gc
gc.collect()


31

In [13]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 1            |        cudaMalloc retries: 4         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   6216 MiB |  13428 MiB |  74259 MiB |  68043 MiB |\n|       from large pool |   6184 MiB |  13396 MiB |  74165 MiB |  67981 MiB |\n|       from small pool |     32 MiB |     34 MiB |     94 MiB |     62 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   6216 MiB |  13428 MiB |  74259 MiB |  68043 MiB |\n|       from large pool |   6184 MiB |  13396 MiB |

In [10]:
#Quantize
# dont run it again
model.quantize(tokenizer, quant_config=quant_config)

/tmp/pip_packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating validation split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (8322 > 4096). Running this sequence through the model will result in indexing errors
AWQ: 100%|██████████| 32/32 [37:08<00:00, 69.63s/it]


In [11]:
# model_quantized = AutoModelForCausalLM.from_pretrained(
#     model,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     quantization_config=quant_config
# )

In [13]:
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

OSError: [Errno 30] Read-only file system: '/data/quantized_model'

### Inference on quantized model

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
quant_path = '/data/quantization-trials/quant-model'
model_path = '/data/quantization-trials/merged-model'

In [14]:
model_id = quant_path
quant_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

def predict_from_quant(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = quant_model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [15]:
import time

In [23]:
# Using quant model
start = time.time()
output1 = predict_from_quant("what is art")
print("time taken is :", time.time()-start)

time taken is : 21.147445678710938


In [24]:
print(output1)

</s>what is art?

Art is a way of expressing one's self. It is a way of expressing one's thoughts, feelings, and ideas. It is a way of expressing one's self through the use of various media.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art forms. The term art is used to describe a wide variety of art forms.

The term art is used to describe a wide variety of art for

### Inference on un-quantized model

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [19]:
model_path = '/data/quantization-trials/merged-model'

In [20]:
model_id = model_path
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

def predict_from_normal(user_query):
    _inputs = tokenizer.encode(user_query, return_tensors="pt").to('cuda')
    outputs = model.generate(input_ids=_inputs, max_length= 1000, pad_token_id=tokenizer.eos_token_id)
    output = tokenizer.decode(outputs[0])
    return output

In [25]:
# Using unquant model
import time
start = time.time()
output2 = predict_from_normal("what is art")
print("time taken is :", time.time()-start)

time taken is : 23.784337282180786


In [26]:
print(output2)

</s>what is art?

Art is a creative expression of the human mind. It is a way of expressing the human mind through the use of various media.

The word art is derived from the Latin word artus, which means "to make". The word art is used in many different contexts, including:

- to describe the beauty of a work of art

- to describe the meaning of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work of art

- to describe the structure of a work 

In [26]:
model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): LayerN

In [27]:
quant_model

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (v_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (q_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
            (out_proj): WQLinear_GEMM(in_features=2048, out_features=2048, bias=True, w_bit=4, group_size=512)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): WQLinear_GEMM(in_features=2048, out